In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
EXECUTABLE_NAME_SCENARIO = 'cos_dist_copy'
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"

# random number generator
rng = np.random.default_rng()

In [12]:
def run_sfe(x, y, y_0, y_1):
    # write the original 2 vectors to a file (second vector used only for verification)
    with open(f"{EXECUTABLE_PATH}/{INPUT_FILE_NAME}", 'w') as f:
        for x_i, y_i in zip(x, y):
            f.write(f"{x_i} {y_i}\n")
            
    # write the shares into separate files
    with open(f"{EXECUTABLE_PATH}/share0.txt", 'w') as f:
        for i in y_0:
            f.write(f"{i}\n")
    with open(f"{EXECUTABLE_PATH}/share1.txt", 'w') as f:
        for i in y_1:
            f.write(f"{i}\n")
            
    # execute the ABY cos sim computation
    output = subprocess.run(CMD_SCENARIO, shell=True, capture_output=True, text=True, cwd=EXECUTABLE_PATH)
    assert (output.returncode == 0) # make sure the process executed successfully
    
    return output

def get_embedding(imagepath):
    return DeepFace.represent(img_path = imagepath, model_name="SFace", enforce_detection=True)[0]["embedding"]

def get_cos_dist_numpy(x, y):
    return 1 - np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

def get_cos_dist_nom(x, y):
    return 1 - np.dot(x, y)
        
def get_two_random_embeddings(same_person):
    """Get two random embeddings of either the same person or two different people out of all the images available"""
    people = os.listdir('lfw') # list of all people that have images
    people_with_multiple_images = [p for p in people if len(os.listdir(f"lfw/{p}")) > 1] # list of people with more than one image in folder
    embedding1, embedding2 = None, None # face embeddings
    while embedding1 is None or embedding2 is None: # try until the chosen images have detectable faces
        try:
            if same_person:
                # same person should have more than one image (we might still end up choosing the same image of that person with prob 1/n, but that's ok)
                person1 = random.choice(people_with_multiple_images)
                person2 = person1
            else:
                # two persons chosen should be different
                person1 = random.choice(people)
                person2 = random.choice([p for p in people if p != person1])
            # get two random images
            img1 = f"lfw/{person1}/{random.choice(os.listdir(f'lfw/{person1}'))}"
            img2 = f"lfw/{person2}/{random.choice(os.listdir(f'lfw/{person2}'))}"
            # try to extract embeddings from both images
            embedding1 = get_embedding(img1)
            embedding2 = get_embedding(img2)
        except Exception as e:
            # failed to detect faces in images, try again
            # print(e)
            pass
    return np.array(embedding1), np.array(embedding2)

def write_two_random_vecs(as_int=False):
    x, y = get_two_random_embeddings(False)
    if as_int:
        x = x.astype(int)
        y = y.astype(int)
    with open(f"{EXECUTABLE_PATH}/{INPUT_FILE_NAME}", 'w') as f:
        for x_i, y_i in zip(x, y):
            f.write(f"{x_i} {y_i}\n")

In [4]:
embedding1 = get_embedding("lfw/George_W_Bush/George_W_Bush_0001.jpg")
embedding2 = get_embedding("lfw/George_W_Bush/George_W_Bush_0002.jpg")
get_cos_dist_numpy(embedding1, embedding2)

0.26386112452051613

In [6]:
norm_emb_1 = embedding1 / np.linalg.norm(embedding1)
norm_emb_2 = embedding2 / np.linalg.norm(embedding2)

In [7]:
embedding1

[-0.6843489408493042,
 0.6780056357383728,
 0.42722856998443604,
 -0.3400259017944336,
 -0.6200863122940063,
 2.514937162399292,
 -1.3317066431045532,
 0.7882114052772522,
 -1.024553894996643,
 -1.109717607498169,
 -2.748328924179077,
 1.0827183723449707,
 -1.0164796113967896,
 0.5706548690795898,
 0.43959739804267883,
 0.3957173228263855,
 -2.0649592876434326,
 -0.35260209441185,
 1.0443885326385498,
 1.276161551475525,
 -1.0018441677093506,
 0.6367229223251343,
 1.031566858291626,
 -0.6151783466339111,
 -0.692867636680603,
 -1.4578611850738525,
 -0.687010645866394,
 0.8452903628349304,
 1.0708965063095093,
 -0.3111672103404999,
 0.8540692329406738,
 -1.3137155771255493,
 0.3787042498588562,
 -2.1721811294555664,
 0.5685135126113892,
 1.2016093730926514,
 0.9706039428710938,
 -1.0975675582885742,
 0.8012404441833496,
 -0.10437052696943283,
 -0.4940742552280426,
 -0.7515782117843628,
 2.545137405395508,
 0.43849530816078186,
 -0.26104050874710083,
 -1.5399278402328491,
 0.9504088163375

In [8]:
norm_emb_1

array([-0.05577931,  0.05526229,  0.03482217, -0.02771453, -0.05054145,
        0.20498529, -0.10854358,  0.06424484, -0.08350844, -0.09044989,
       -0.22400838,  0.08824926, -0.08285033,  0.04651244,  0.03583032,
        0.03225378, -0.16830889, -0.02873958,  0.0851251 ,  0.10401626,
       -0.08165744,  0.05189745,  0.08408005, -0.05014142, -0.05647365,
       -0.11882607, -0.05599626,  0.06889719,  0.08728569, -0.02536234,
        0.06961273, -0.10707718,  0.03086709, -0.17704823,  0.0463379 ,
        0.09793972,  0.07911113, -0.08945957,  0.0653068 , -0.00850694,
       -0.04027057, -0.06125898,  0.20744683,  0.03574049, -0.02127666,
       -0.12551509,  0.07746509,  0.05844309, -0.15284203, -0.0653634 ,
       -0.04201705, -0.06837048, -0.08379623,  0.0950596 , -0.00335818,
       -0.00347915, -0.11427014, -0.06078857,  0.09876206, -0.00900666,
        0.04687155,  0.14139215, -0.01919006,  0.02727786, -0.06001622,
        0.07840079,  0.05644274, -0.05244616, -0.01176201,  0.02

In [11]:
get_cos_dist_numpy(norm_emb_1, norm_emb_2)

0.26386112452051613

In [13]:
get_cos_dist_nom(norm_emb_1, norm_emb_2)

0.26386112452051613

In [5]:
result = DeepFace.verify(img1_path = "lfw/George_W_Bush/George_W_Bush_0001.jpg", img2_path = "lfw/George_W_Bush/George_W_Bush_0002.jpg", model_name="SFace")
result

{'verified': True,
 'distance': 0.26386112452051613,
 'threshold': 0.593,
 'model': 'SFace',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 70, 'y': 72, 'w': 108, 'h': 108},
  'img2': {'x': 69, 'y': 70, 'w': 113, 'h': 113}},
 'time': 0.07}

In [24]:
# 1st try with real data

# Get two embeddings of images that we will be comparing.
a, b = get_two_random_embeddings(same_person=False)

# This is how they look like raw
a

array([-8.83551598e-01, -4.29614931e-01, -6.73904359e-01,  7.80343413e-01,
        1.28018951e+00, -4.28495675e-01,  1.52694774e+00,  2.06022882e+00,
       -7.50259876e-01, -1.80774271e-01,  4.34262246e-01,  1.49772123e-01,
        3.83244097e-01,  7.17375159e-01, -1.60901830e-01, -8.18099603e-02,
       -2.44410992e+00,  1.02034438e+00, -2.94784904e+00,  3.05196786e+00,
       -5.64855158e-01,  7.30196357e-01,  4.58130330e-01, -7.65918493e-01,
        8.64430487e-01, -1.35592782e+00, -3.59060708e-04, -8.11428308e-01,
        1.53911710e-01, -6.08670354e-01, -4.65954036e-01, -1.04143798e-01,
       -3.69812965e-01,  8.91645849e-01, -2.34297186e-01,  8.03180933e-01,
        3.29210162e-02, -9.61613178e-01,  1.06024468e+00,  6.21587753e-01,
        3.88016135e-01,  8.74610782e-01, -1.05808020e+00, -8.69417906e-01,
        2.71820396e-01,  8.30140352e-01, -4.21035647e-01, -4.98189718e-01,
       -8.12705874e-01,  5.96207500e-01,  1.07833791e+00, -1.00928962e+00,
        4.13474470e-01,  

In [32]:
# Make the shares
# x is the captured face, y is the face in the database
# First, scale the values up by 10 000, then get rid of the decimal part then cast to int

x = (a * 100).round().astype(int)
y = (b * 100).round().astype(int)

# so far so good
x

array([ -88,  -43,  -67,   78,  128,  -43,  153,  206,  -75,  -18,   43,
         15,   38,   72,  -16,   -8, -244,  102, -295,  305,  -56,   73,
         46,  -77,   86, -136,    0,  -81,   15,  -61,  -47,  -10,  -37,
         89,  -23,   80,    3,  -96,  106,   62,   39,   87, -106,  -87,
         27,   83,  -42,  -50,  -81,   60,  108, -101,   41,  109,   -5,
       -146,  -17,  -63,   20,   57,  173,    7,  -67,   38,   67,  -13,
         21, -125,   67,   90,   41,  -48,   -8,   -3,   25,  156,  -68,
        -57,  -29,   47,  -33,   49,   56,   57, -155,   89,   41, -120,
         -3,  154,  112,  -79,  -15,  -21,  -39, -146, -124,   41,  -88,
        143,   24,   -5,  -11,   46,   27,  -87,  -68,   26,  -37,  -59,
         81, -141,    7,  117,  -17,  -73,   -7,   35,  -33,   -5,   50,
        -20,   67, -112,   23,  151,   60,  149])

In [33]:
y

array([ 52, -33,   9, -41,  77,   1, -15, -36, -12,   6, 139, -21, 122,
       -48, -14, -61, -43, -21,   4,  26,  -2, -50, -16, -53, -25,  91,
        57,  19, -24,  11, -12,   9, -15,  25, -50, -15,  18,  42,   0,
       -29, -67,  40,  40,  -9,  27,  26, -27, -46,   9,  -4,  93, -43,
        11,  17,   8,  88,  11,  57, -34, -35, -42, -25, -55,  12,  19,
        -8,  41,  33,  -3,  17, -55,   6,  41, -29, -15, -31,  34,  76,
       -24,  10,  58,   8,  79, -33,  -8,  64,  17,  32,  38, 101,  24,
       135,  58,  35,  44, -14, -27, -17, -20,  34,   0,  57,  53,  22,
       -58,  43,  45,  -9, -30,  -6,  59,  11,  59, -21,  12,  69, -69,
       -12,  39,  -4, -55, 122, -15,  14,  40, -25,  18,  39])

In [35]:
# Now create the shares

# random nonces, values in the same range as the embeddings after scaling
r = rng.integers(-300, 300, 128)
r

array([-253,  290, -281,  -37,   62, -238, -115, -195,   86, -153,   59,
       -135,   39, -192, -258, -278,  134,  280,  182, -245, -293,  -57,
         44, -178,  253,  143,  187,    2,   43,  265,  210,  162,  -71,
       -139, -181,  173, -160,  159,  -53, -260,   34,    5,  -46,  238,
       -152,   26,  288, -209, -285,   64,  184, -225, -174, -151,  109,
         45, -130, -182, -282,  129, -143,   96,  162, -225,   22, -243,
       -132,    6,  217,  -20,  166, -256,  127,   62,   -5, -128, -193,
         -4,  272,  288, -162,  -60,  132,  228,  219, -275,  249,  261,
         -3,   72, -276,  178,   86,  178,  131,  234, -196,  -69,  -68,
         64,  -50,  269,  178,  -40,  137, -149, -141,  287,  291,  138,
       -290,   56,   52,    6,   10,   29,    5,   74,  274,  -65,   -1,
        267,  -33,  -60,  217,  254,  262,  259])

In [36]:
# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

y_0

array([-201, -259, -274,   12,  115, -237,  124,  225,  -94, -159,  176,
        146,   93,  144,  268,  297, -173, -269,  178, -239,  293,    9,
        -36,  133, -230,  212,  130,   17,  -61,  258, -218,  171,   72,
       -148,  133, -164, -142,  181,  -53,  287,  -97,   45,   -6, -231,
       -141,    0, -315,  253, -278,  -68,  229,  202, -167, -136,  101,
        117, -139, -141,  312, -164,  167, -121, -149, -237,    5,  245,
       -171,   39, -220,   -3, -145, -250,   86,  -35,   10,   97, -227,
        -80, -264,  298, -156,  -52,  203, -197, -221, -339,  232,  293,
        -37,   45, -268,   53,  108,  145,  175, -232,  217,   84,   80,
         98,  -50,  308,  135,  -50, -177, -192, -162, -280, -319, -144,
       -283,   51,   15,  -19,    6,   88,  -66,  -66,  309,   67,   54,
        369,   46,  -54,  241, -231,  276,  292])

In [37]:
# To check that the xoring works, we can XOR y_0 with y_1 and we expect to obtain y

y == np.bitwise_xor(y_0, y_1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [38]:
# Let's run the ABY code (I am providing y for verification, it's not needed nor used in the circuit)

output = run_sfe(x, y, y_0, y_1)

In [40]:
"""inspect the results
After the statistics from ABY I am printing:
- the input x,y and the share (in this case output is from the server,so we see share y_1)
- the verification results and the circuit results
"""
# Unfortunately, circuit result is incorrect.
print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
s_product nvals: 128
s_product bitlen: 32
Online time is distributed as follows: 
Bool: local gates: 14.1670000000, interactive gates: 12.2380000000, layer finish: 6.6670000000
Yao: local gates: 0.1480000000, interactive gates: 0.1310000000, layer finish: 0.0530000000
Yao Rev: local gates: 0.1390000000, interactive gates: 0.1450000000, layer finish: 0.0780000000
Arith: local gates: 0.1770000000, interactive gates: 0.1310000000, layer finish: 0.1260000000
SPLUT: local gates: 0.1620000000, interactive gates: 0.1340000000, layer finish: 0.7900000000
Communication: 38.9110000000

Complexities: 
Boolean Sharing: ANDs: 1039177 (1-bit) ; Depth: 589
Total Vec AND: 1039177
Total Non-Vec AND: 1039177
XOR vals: 3891143 gates: 98669
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 12288, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing: OT-gates: Total OT g

In [11]:
# Since we are getting 'inf' values, I scale by a smaller amount and see if that helps
# Let's go extreme and round the floats

x = a.round().astype(int)
y = b.round().astype(int)


x

array([ 1, -1,  0, -2,  1, -1, -2, -1, -2, -1, -1,  0,  1,  1,  1, -1,  3,
       -1,  1,  0, -1,  1,  0,  1,  2,  0, -1, -1, -1,  0,  1,  0, -1,  1,
        1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -2, -1, -1, -1,  0,
        1, -1, -1, -1,  1,  1,  0,  0,  1,  0,  0, -1, -1, -1,  0,  1,  1,
       -1,  1,  2,  1,  1,  0,  2,  1,  0,  1, -1,  0,  0, -1,  0, -2,  0,
        2,  1,  1,  0,  1, -1,  0, -1,  0,  1, -1,  0,  1,  1,  0,  1, -2,
        1, -2,  0,  1, -1,  0, -1, -1,  0,  0,  0, -2, -2, -1,  0,  0, -1,
        0,  1,  0,  2,  1, -1,  0,  0,  1])

In [12]:
# Now create the shares, everything the same way as above

# random nonces, values in the same range as the embeddings after scaling
r = rng.integers(-3, 3, 128)

# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

# To check that the xoring works, we can XOR y_0 with y_1 and we expect to obtain y
y == np.bitwise_xor(y_0, y_1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [13]:
# Let's run the ABY code (I am providing y for verification, it's not needed nor used in the circuit)

output = run_sfe(x, y, y_0, y_1)


In [14]:
# inspect the results

# Unfortunately, circuit result is incorrect.
print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
s_product nvals: 128
s_product bitlen: 32
Online time is distributed as follows: 
Bool: local gates: 16.5100000000, interactive gates: 15.0440000000, layer finish: 7.4260000000
Yao: local gates: 0.1450000000, interactive gates: 0.1740000000, layer finish: 0.0440000000
Yao Rev: local gates: 0.1600000000, interactive gates: 0.1530000000, layer finish: 0.0640000000
Arith: local gates: 0.1640000000, interactive gates: 0.1310000000, layer finish: 0.1360000000
SPLUT: local gates: 0.1440000000, interactive gates: 0.1390000000, layer finish: 0.8130000000
Communication: 43.7520000000

Complexities: 
Boolean Sharing: ANDs: 1039177 (1-bit) ; Depth: 589
Total Vec AND: 1039177
Total Non-Vec AND: 1039177
XOR vals: 3891143 gates: 98669
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 12288, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing: OT-gates: Total OT g

In [41]:
# I discovered by accident that if the vectors are only 0s and 1s then the circuit works as expected...
# see example

# some arrays with only 0s or 1s
x = rng.integers(0, 100, 128)
y = rng.integers(0, 100, 128)
r = rng.integers(0, 100, 128)

# create the shares in the same way as before

# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

output = run_sfe(x, y, y_0, y_1)

print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
x: 7 | y: 5
x: 96 | y: 71
x: 57 | y: 50
x: 69 | y: 79
x: 77 | y: 47
x: 91 | y: 57
x: 77 | y: 84
x: 99 | y: 53
x: 20 | y: 39
x: 10 | y: 93
x: 4 | y: 98
x: 70 | y: 53
x: 80 | y: 4
x: 50 | y: 94
x: 19 | y: 11
x: 78 | y: 49
x: 54 | y: 58
x: 37 | y: 31
x: 68 | y: 46
x: 88 | y: 67
x: 56 | y: 54
x: 7 | y: 86
x: 85 | y: 81
x: 1 | y: 49
x: 37 | y: 84
x: 90 | y: 7
x: 62 | y: 45
x: 24 | y: 14
x: 79 | y: 91
x: 7 | y: 29
x: 82 | y: 79
x: 55 | y: 11
x: 18 | y: 61
x: 98 | y: 11
x: 32 | y: 86
x: 38 | y: 18
x: 56 | y: 83
x: 34 | y: 50
x: 22 | y: 22
x: 77 | y: 42
x: 29 | y: 66
x: 79 | y: 69
x: 83 | y: 55
x: 47 | y: 92
x: 64 | y: 72
x: 51 | y: 15
x: 79 | y: 54
x: 72 | y: 69
x: 59 | y: 70
x: 79 | y: 56
x: 14 | y: 26
x: 99 | y: 63
x: 55 | y: 91
x: 47 | y: 71
x: 26 | y: 57
x: 25 | y: 68
x: 10 | y: 26
x: 8 | y: 33
x: 36 | y: 34
x: 33 | y: 42
x: 63 | y: 31
x: 31 | y: 9
x: 63 | y: 10
x: 40 | y: 84
x: 66 | y: 18
x: 39 | y: 69
x: 7 | y: 15
x: 12 | y: 73
x: 48 | 

In [42]:
get_cos_dist_numpy(x,y)

0.2344976457505803